In [1]:
import sys
sys.path.append('../personal_pkgs/')
import pickle
import pandas as pd
import oop_func as func
import personal_pkg as ref
import parsing_func as pars
import os
import re
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Markdown
from ordered_set import OrderedSet

In [2]:
base_df = pd.read_csv('base_df.csv')
fm_base_df = pd.read_csv('fm_base_df.csv')
f_base_df = pd.read_csv('f_base_df.csv')
sc_base_df = pd.read_csv('sc_base_df.csv')
dr_base_df = pd.read_csv('dr_base_df.csv')

In [3]:
def return_multiple_author(base_df):
    testing_df = base_df[base_df['author'].notnull()]
    author = testing_df['author'].tolist()
    author =\
    [i.replace('[Provider:earticle]  ','') if '[provider:earticle]' in i.lower() else i for i in author]
    testing_df['journal'] = author
    return testing_df.loc[:,['author','journal','paper','year','from']]

In [4]:
base_df = return_multiple_author(base_df)
fm_base_df = return_multiple_author(fm_base_df)
f_base_df = return_multiple_author(f_base_df)
sc_base_df = return_multiple_author(sc_base_df)
dr_base_df = return_multiple_author(dr_base_df)

In [5]:
fm_base_df['year'] = fm_base_df['year'].apply(lambda x : str(x).split('.')[0])
f_base_df['year'] = f_base_df['year'].apply(lambda x : str(x).split('.')[0])
sc_base_df['year'] = sc_base_df['year'].apply(lambda x : str(x).split('.')[0])
dr_base_df['year'] = dr_base_df['year'].apply(lambda x : str(x).split('.')[0])

In [6]:
counter_df = pd.DataFrame.from_dict(Counter(base_df['author']),orient='index',columns=['count']).sort_values('count',ascending=False)
counter_df.head(10)

,count
Fama E. F. and K. R. French,295
"19’‘”에 '""… ‘ N. o. ’ . '"" s‘· ’……… _""_ .A Cι SIUdy of rbo ’._ ..，띠r생@ι-.'.-ι",280
Black F.,131
Cifuentes A. and G. Katsaros 2007 The One-Factor Gaussian Copula Applied to CDOs: Just Say NO (or if you see a correlation smile she is laughing at your,114
Jegadeesh N. and S. Titman,83
Jensen M. and W. Meckling,82
Carhart Mark M.,82
고봉찬 김진우,70
Bollerslev T.,68
Fama Eugene F. Kenneth R. French,66


In [7]:
catch_author_ls = counter_df[counter_df['count'] > 3].index.tolist()
print(len(catch_author_ls))
catch_author_ls[:5]

1533


['Fama E. F. and K. R. French ',
 '19’‘”에 \'"…   ‘ N. o. ’ . \'" s‘· ’……… _"_ .A Cι SIUdy of rbo ’._ ..，띠r생@ι-.\'.-ι ',
 'Black F. ',
 'Cifuentes A. and G. Katsaros 2007 The One-Factor Gaussian Copula Applied to CDOs: Just Say NO (or if you see a correlation smile she is laughing at your ',
 'Jegadeesh N. and S. Titman ']

In [8]:
catch_author_ls_copy = catch_author_ls.copy()
tuned_catch_author_ls = []
excepted_ls = []
similar_match_ls = []

for i in range(5) : 
    print(i)
    for idx1,val1 in enumerate(catch_author_ls_copy) : 
        for idx2,val2 in enumerate(catch_author_ls_copy) :
            if idx1 != idx2 : 
                v1 = set(val1.split(' '))
                v2 = set(val2.split(' '))
                try : 
                    if len(set(v1).intersection(v2)) / ((len(v1) + len(v2))/2) > 0.85 : 
                        similar_match_ls.append((val1,val2))
                        if len(v1) > len(v2) :
                            catch_author_ls_copy.pop(catch_author_ls_copy.index(val2))
                            excepted_ls.append((val2))
                        else :
                            catch_author_ls_copy.pop(catch_author_ls_copy.index(val1))
                            excepted_ls.append(((val1)))                    
                except : pass

len(catch_author_ls) , len(catch_author_ls_copy) , len(excepted_ls)

0
1
2
3
4


(1533, 1295, 238)

In [9]:
ready_to_target_dict = dict(zip(catch_author_ls_copy,[[] for i in range(len(catch_author_ls_copy))]))
append = 0
naive_append = 0

for idx1,val1 in enumerate(base_df['author'].tolist()) : 
    if idx1 % 4000 == 0 : print(idx1,end='\r')
    for idx2,val2 in enumerate(ready_to_target_dict.keys()) :
        
        if type(val1) == str:
            paper_name = val1.lower()
            target_name = val2.lower()

            paper_name_to_word = set(paper_name.split(' '))
            target_name_to_word = set(target_name.split(' '))
            if len(paper_name_to_word.intersection(target_name_to_word)) / \
            ((len(paper_name_to_word) + len(target_name_to_word))/2) > 0.85 : 
                ready_to_target_dict[val2].append(val1)
                append += 1

In [10]:
pickle_out = open("author_ready_to_target_dict.pickle","wb")
pickle.dump(ready_to_target_dict, pickle_out)
pickle_out.close()

In [11]:
def parse_multiple_to_single(author) :
    if type(author) == str :
        author = author.strip()
        if re.search('[가-힣]',author) : 
            return re.split(' ',author)
        elif re.search('[a-zA-Z]',author) : 
            return re.split('and',author)
    else : return author

In [12]:
def return_df_author_frequency_table_under_journal(base_df):
    final_ls = []
    process_ls = []

    for idx1 , val in enumerate(base_df['author']) : 
        process_ls = []
        for idx2, key_ in enumerate(list(ready_to_target_dict.keys())) : 
            if val in ready_to_target_dict.get(key_) : 
                process_ls.append(key_)
        if type(val) == str :
            if len(process_ls) == 1 :
                final_ls.append(process_ls[0])
            elif len(process_ls) == 0 :
                final_ls.append(val)
            else : final_ls.append(val)
        else : final_ls.append(None)

    if len(final_ls) == len(base_df) : 
        base_df['new_author'] = final_ls
    else : print('error')
    
    print("lower uniqueness means better performance, orgin uniqueness : {} , revised uniqueness : {}"\
          .format(len(set(base_df['author'])),len(set(final_ls))))
    return base_df

In [13]:
base_final_df = return_df_author_frequency_table_under_journal(base_df)
fm_final_df = return_df_author_frequency_table_under_journal(fm_base_df)
f_final_df = return_df_author_frequency_table_under_journal(f_base_df)
sc_final_df = return_df_author_frequency_table_under_journal(sc_base_df)
dr_final_df = return_df_author_frequency_table_under_journal(dr_base_df)

lower uniqueness means better performance, orgin uniqueness : 12325 , revised uniqueness : 11647
lower uniqueness means better performance, orgin uniqueness : 6933 , revised uniqueness : 6556
lower uniqueness means better performance, orgin uniqueness : 4849 , revised uniqueness : 4562
lower uniqueness means better performance, orgin uniqueness : 3057 , revised uniqueness : 2775
lower uniqueness means better performance, orgin uniqueness : 1937 , revised uniqueness : 1844


In [14]:
base_final_df['single_author'] = base_final_df['new_author'].apply(lambda x : parse_multiple_to_single(x))
fm_final_df['single_author'] = fm_final_df['new_author'].apply(lambda x : parse_multiple_to_single(x))
f_final_df['single_author'] = f_final_df['new_author'].apply(lambda x : parse_multiple_to_single(x))
sc_final_df['single_author'] = sc_final_df['new_author'].apply(lambda x : parse_multiple_to_single(x))
dr_final_df['single_author'] = dr_final_df['new_author'].apply(lambda x : parse_multiple_to_single(x))

In [15]:
base_testing_df = base_final_df[base_final_df['single_author'].notnull()]
fm_testing_df = fm_final_df[fm_final_df['single_author'].notnull()]
f_testing_df = f_final_df[f_final_df['single_author'].notnull()]
sc_testing_df = sc_final_df[sc_final_df['single_author'].notnull()]
dr_testing_df = dr_final_df[dr_final_df['single_author'].notnull()]

In [16]:
base_testing_ls = [i for i in base_testing_df['single_author'] if len(i) < 5]
fm_testing_ls = [i for i in fm_testing_df['single_author'] if len(i) < 5]
f_testing_ls = [i for i in f_testing_df['single_author'] if len(i) < 5]
sc_testing_ls = [i for i in sc_testing_df['single_author'] if len(i) < 5]
dr_testing_ls = [i for i in dr_testing_df['single_author'] if len(i) < 5]

In [17]:
base_single_author_ls = [j.strip() for i in base_testing_ls for j in i if j]
fm_single_author_ls = [j.strip() for i in fm_testing_ls for j in i if j]
f_single_author_ls = [j.strip() for i in f_testing_ls for j in i if j]
sc_single_author_ls = [j.strip() for i in sc_testing_ls for j in i if j]
dr_single_author_ls = [j.strip() for i in dr_testing_ls for j in i if j]

In [18]:
author_counter_df = pd.DataFrame.from_dict(Counter(base_single_author_ls),orient='index',columns=['count']).sort_values('count',ascending=False)
author_counter_df.reset_index(inplace=True)
display(author_counter_df.head(10))
author_counter_df.to_csv('author_counter_df.csv',index=False)

,index,count
0,Fama E. F.,465
1,K. R. French,387
2,Jensen M. C.,280
3,Black F.,248
4,Fama Eugene F.,173
5,W. H. Meckling,162
6,고봉찬,153
7,French Kenneth R.,147
8,고광수,135
9,박경서,127


In [19]:
display(pd.DataFrame.from_dict(Counter(fm_single_author_ls),orient='index',columns=['count']).sort_values('count',ascending=False).head(20))
display(pd.DataFrame.from_dict(Counter(f_single_author_ls),orient='index',columns=['count']).sort_values('count',ascending=False).head(20))
display(pd.DataFrame.from_dict(Counter(sc_single_author_ls),orient='index',columns=['count']).sort_values('count',ascending=False).head(20))
display(pd.DataFrame.from_dict(Counter(dr_single_author_ls),orient='index',columns=['count']).sort_values('count',ascending=False).head(20))

,count
Fama E. F.,128
K. R. French,93
Jensen M. C.,91
H,49
권택호,45
W. H. Meckling,44
R. Schwartz,44
a P.,44
G. Katsaros 2007 The One-Factor Gaussian Copula Applied to CDOs: Just Say NO (or if you see a correlation smile she is laughing at your,43
Cifuentes A.,43


,count
Fama E. F.,64
K. R. French,46
엄경식,45
[Provider:earticle],39
Jensen M. C.,38
박경서,34
고봉찬,34
Fama Eugene F.,30
(In,29
장하성,29


,count
Fama E. F.,230
K. R. French,215
Jensen M. C.,150
Black F.,145
W. H. Meckling,107
Fama Eugene F.,85
Carhart Mark M.,76
M. Scholes,73
French Kenneth R.,68
R. M. Stulz,62


,count
Black F.,43
Fama E. F.,43
D. Madan,39
Bakshi G. N. Kapadia,35
K. R. French,33
Sutcliffe C.,33
김,30
윤선중,30
솔,28
이재하,27
